In [8]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer


# Specify the file path
# file_path = "/Users/p.mittal/Library/Mobile Documents/com~apple~CloudDocs/Roux/Courses/NLP/Assignments/Assignment 1 BOW/Subset_patient_notes.csv"

# Read the CSV file using pandas
notes = pd.read_csv('/content/patient_notes.csv')
# notes = notes.sample(frac=0.01, random_state=1)

# Process the data
print(notes.shape)
notes.head()

notes["case_num"].value_counts()


ParserError: Error tokenizing data. C error: Expected 3 fields in line 37158, saw 5


In [ ]:
# ---------------------------------------
# ---------------------------------------
# Step 2: Cretae a customized Tokenizer
# ---------------------------------------
# ---------------------------------------

from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import word_tokenize
import re


# ---------------------------------------
# Initialize stemmer and lemmatizer
# ---------------------------------------
# i. Case Conversion
notes['pn_history'] = notes['pn_history'].str.lower()

# ii. Removing Punctuation and Special Characters
notes['pn_history'] = notes['pn_history'].str.replace(r'[^\w\s]', '', regex=True)


stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

def custom_tokenizer(text):
    # Remove special characters and numbers
    text = re.sub(r'\W|\d', ' ', text)
#     # Define a regular expression pattern to identify and preserve age-related formats
#     age_pattern = r'\b\d{1,3}(?:(?:[-/]?[Yy](?:[Oo]?|[Ee][Aa][Rr]\s?[Oo][Ll][Dd]))|(?:[-/]?\d{1,3}(?:[Yy]\.?[Oo])?))\b'

#     # Replace all non-age-related numbers with a space
#     text = re.sub(r'\b(?!(?:[-/]?[Yy](?:[Oo]?|[Ee][Aa][Rr]\s?[Oo][Ll][Dd]))|(?:[-/]?\d{1,3}(?:[Yy]\.?[Oo])?))\d{1,3}\b', ' ', text)

#     # Remove special characters but keep numbers
#     text = re.sub(r'[^\w\s\d]', ' ', text)


    # Tokenize text
    tokens = word_tokenize(text)

    # Remove single-character tokens
    tokens = [token for token in tokens if len(token) > 2]

    # Apply stemming and lemmatization
#     stemmed_tokens = [stemmer.stem(token) for token in tokens]
#     lemmatized_tokens = [lemmatizer.lemmatize(token) for token in stemmed_tokens]
#     stemmed_tokens = [stemmer.stem(token) for token in tokens]
#     lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]

    return tokens

In [ ]:
import collections

# Create a frequency dictionary
def create_frequency_dictionary(tokens):
    frequency_dict = collections.Counter(tokens)
    return frequency_dict

# Create a bigram dictionary
def create_bigram_dictionary(tokens):
    bigram_dict = collections.Counter(zip(tokens, tokens[1:]))
    return bigram_dict

# Tokenize and preprocess the 'pn_history' column
tokenized_pn_history = notes['pn_history'].apply(custom_tokenizer)

# Flatten the list of tokenized notes into a single list of tokens
all_tokens = [token for sublist in tokenized_pn_history for token in sublist]

# Create frequency dictionary
frequency_dictionary = create_frequency_dictionary(all_tokens)

# Create bigram dictionary
bigram_dictionary = create_bigram_dictionary(all_tokens)

# Save frequency dictionary to a text file
with open("frequency_dictionary.txt", "w") as f:
    for word, count in frequency_dictionary.items():
        f.write(f"{word}: {count}\n")

# Save bigram dictionary to a text file
with open("bigram_dictionary.txt", "w") as f:
    for bigram, count in bigram_dictionary.items():
        f.write(f"{bigram[0]} {bigram[1]}: {count}\n")


In [ ]:
import symspellpy
import re

# Create a SymSpell object
symspell = symspellpy.SymSpell()

# Load a frequency dictionary
dictionary_path = "frequency_dictionary.txt"
bigram_path = "bigram_dictionary.txt"

# Load the dictionary and bigram data
symspell.load_dictionary(dictionary_path, term_index=0, count_index=1)
symspell.load_bigram_dictionary(bigram_path, term_index=0, count_index=2)

# Function for spelling correction using SymSpell
def correct_spelling_symspell(text):
    # Tokenize the input text using regular expressions (split on non-word characters)
    words = re.findall(r'\w+', text)

    # Correct the spelling of each word using SymSpell
    corrected_words = [symspell.lookup(word, symspellpy.Verbosity.CLOSEST, max_edit_distance=2, include_unknown=True)[0].term for word in words]

    # Remove any colons that might have been added during correction
    corrected_words = [word.replace(':', '') for word in corrected_words]

    # Join the corrected words back into a text with spaces
    corrected_text = ' '.join(corrected_words)

    return corrected_text

# Apply the spelling correction function to the 'pn_history' column
notes['pn_history'] = notes['pn_history'].apply(correct_spelling_symspell)
print(notes['pn_history'])



In [ ]:


# ---------------------------------------
# Apply customized tokenizer
# ---------------------------------------

vectorizer = CountVectorizer(tokenizer=custom_tokenizer)


# ---------------------------------------
# Fit and transform the 'pn_history' column
# ---------------------------------------
pn_history_vector = vectorizer.fit_transform(notes['pn_history'])

# ---------------------------------------
# Print the shape of the vectorized 'pn_history' column
# ---------------------------------------
print(pn_history_vector.shape)

# ---------------------------------------
# Get the feature names (tokens)
# ---------------------------------------
feature_names = vectorizer.get_feature_names()
print("Number of features:", feature_names[0:10])


# ---------------------------------------
# Parameters of CountVectorizer()
# ---------------------------------------
CountVectorizer()

(421, 3744)
Number of features: ['_ve', 'aaf', 'abd', 'abdmoninal', 'abdomen', 'abdomianl', 'abdominal', 'abdominalpelvic', 'abdoninal', 'able']


/Users/gaurav/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


CountVectorizer()

In [ ]:
# ---------------------------------------
# ---------------------------------------
# Step 3: Modifying the Stop Words list
# ---------------------------------------
# ---------------------------------------

# ---------------------------------------
# Print list of Stop words in CountVectorizer()
# ---------------------------------------

stopwords = CountVectorizer(stop_words='english').get_stop_words()
len(stopwords)



# ------------------------------------------------------------------------------
# Adding words to the stop words list
# (Note the choice of additional stop words is only for illustration)
# ------------------------------------------------------------------------------

# nstopwords = list(stopwords) + ['abdomin', 'abdomen', 'abdonmin','age']
nstopwords = list(stopwords)
print(len(stopwords))
print(len(nstopwords))
print(nstopwords)

# --------------------------------------------------------
# repeating the process with the new list of stopwords
# --------------------------------------------------------

vectorizer2 = CountVectorizer(tokenizer=custom_tokenizer, stop_words=nstopwords)


# Fit and transform the 'pn_history' column
pn_history_vector = vectorizer2.fit_transform(notes['pn_history'])

# Get the feature names (tokens)
feature_names2 = vectorizer2.get_feature_names()


print("First few ORIGINAL words:", feature_names[0:100])

print("---------")

print("First few words after new stop list:", feature_names2[0:100])




318
318
['these', 'cry', 'your', 'become', 'mostly', 'whereas', 'over', 'why', 'system', 'few', 'take', 'which', 'anyhow', 'serious', 'ltd', 'none', 're', 'side', 'do', 'alone', 'them', 'will', 'move', 'must', 'either', 'cant', 'there', 'to', 'describe', 'in', 'what', 'empty', 'this', 'no', 'except', 'latterly', 'where', 'not', 'something', 'anyone', 'might', 'name', 'towards', 'is', 'against', 'de', 'wherein', 'anyway', 'only', 'upon', 'i', 'both', 'hers', 'whether', 'up', 'least', 'sometimes', 'therefore', 'eg', 'detail', 'via', 'until', 'twenty', 'enough', 'once', 'are', 'never', 'because', 'could', 'beyond', 'for', 'others', 'thereupon', 'hundred', 'by', 'less', 'my', 'two', 'thick', 'five', 'as', 'already', 'at', 'ten', 'ours', 'became', 'everyone', 'onto', 'everywhere', 'and', 'whither', 'whenever', 'everything', 'that', 'thus', 'fifty', 'ourselves', 'while', 'too', 'the', 'together', 'throughout', 'fire', 'again', 'how', 'can', 'he', 'else', 'amoungst', 'being', 'done', 'many', 

In [ ]:
# ...
# ...

from sklearn.feature_extraction.text import TfidfVectorizer

# Use TfidfVectorizer with custom tokenizer, stop words, and n-grams
vectorizer = TfidfVectorizer(
    tokenizer=custom_tokenizer,
    stop_words=stopwords,
    ngram_range=(1, 1)  # Include unigrams
#     max_df=0.85,         # Ignore terms that have a document frequency higher than 85%
#     min_df=5,            # Ignore terms that have a document frequency lower than 2 documents
#     max_features=10000   # Only consider the top 10,000 features ordered by term frequency across the corpus
)

# Fit and transform the 'pn_history' column
pn_history_vector = vectorizer.fit_transform(notes['pn_history'])

# Print the shape of the vectorized 'pn_history' column
print("Shape of DTM:", pn_history_vector.shape)

# Get the feature names (tokens)
feature_names = vectorizer.get_feature_names_out()

# Print the number of features (tokens)
print("Number of features:", len(feature_names))

with open('feature_names.txt', 'w') as file:
    file.write('\n'.join(feature_names))

# Print the first 100 features
print("First 100 features:", feature_names[:100])

# Describe the DTM from the dataframe (df) perspective
# Number of non-zero elements in the DTM
non_zero_elements = pn_history_vector.getnnz()
print("Number of non-zero elements:", non_zero_elements)


# size_in_bytes = pn_history_vector.toarray().nbytes
print("Size in bytes:", size_in_bytes)

# Sparsity of the DTM
sparsity = 1.0 - (non_zero_elements / (pn_history_vector.shape[0] * pn_history_vector.shape[1]))
print("Sparsity of DTM:", sparsity)

# Create DTM with columns as tokens and rows as documents
pn_history_df_tfidf = pd.DataFrame(pn_history_vector.toarray(), columns=feature_names)
print(pn_history_df_tfidf.head())


Shape of DTM: (421, 3566)
Number of features: 3566
First 100 features: ['_ve' 'aaf' 'abd' 'abdmoninal' 'abdomen' 'abdomianl' 'abdominal'
 'abdominalpelvic' 'abdoninal' 'able' 'abnormal' 'abnormalities'
 'abnormality' 'abnormals' 'abodminal' 'abominal' 'abortions' 'aborts'
 'abovce' 'abrupt' 'absent' 'abuse' 'abuses' 'academically' 'accasionaly'
 'accident' 'accidnet' 'accompainied' 'accompanied' 'accompoanied' 'ace'
 'acetaminophen' 'ache' 'aches' 'achey' 'acheycrampy' 'achieved'
 'achiness' 'aching' 'achy' 'acid' 'acitve' 'acive' 'acknowledged'
 'acloholmo' 'acne' 'acompanied' 'acting' 'active' 'activemonogamous'
 'activieis' 'activites' 'activities' 'activitiesdecreased' 'activity'
 'actually' 'acute' 'acutley' 'acvtie' 'aday' 'added' 'adderal' 'adderall'
 'adderral' 'addies' 'addition' 'additionally' 'additonal' 'aderol'
 'aderral' 'adls' 'admin' 'administrative' 'administrator' 'admistrator'
 'admit' 'admits' 'admitted' 'adopted' 'adversion' 'advil' 'advised'
 'affected' 'affecting

In [ ]:
## Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import TruncatedSVD
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.ensemble import IsolationForest


# ML Pipeline
# a. Normalization
scaler = StandardScaler(with_mean=False)  # with_mean=False is necessary for sparse data
normalized_data = scaler.fit_transform(pn_history_vector)


# Outlier Detection using Isolation Forest
iso_forest = IsolationForest(n_estimators=300, contamination=0.02, random_state=0)
outliers = iso_forest.fit_predict(normalized_data)

# Identifying the rows that are outliers
outlier_indices = np.where(outliers == -1)[0]

# Count the number of outliers
num_outliers = np.sum(outliers == -1)
print(num_outliers)

# Get the number of rows in the sparse matrix
num_rows = normalized_data.shape[0]

# Calculate the percentage of outliers
percentage_outliers = (num_outliers / num_rows) * 100


print(f"Percentage of outliers: {percentage_outliers:.4f}%")


#remove outliers from your DataFrame:
# notes_filtered = notes.drop(index=outlier_indices)



In [ ]:
import matplotlib.pyplot as plt

# Example with TruncatedSVD
svd = TruncatedSVD(n_components=8000)  # Start with a larger number of components
reduced_data = svd.fit_transform(normalized_data)

# Plot the cumulative explained variance
plt.figure(figsize=(10,6))
plt.plot(np.cumsum(svd.explained_variance_ratio_))
plt.xlabel('Number of components')
plt.ylabel('Cumulative explained variance')
plt.show()


In [ ]:
# b. Dimension Reduction
n_components = 6000  # Number of components to keep
svd = TruncatedSVD(n_components=n_components)
reduced_data = svd.fit_transform(normalized_data)
print(f"Total variance explained: {np.sum(svd.explained_variance_ratio_):.2f}")

In [ ]:
# c. Clustering
n_clusters = 7  # Number of clusters
kmeans = KMeans(n_clusters=n_clusters, random_state=0)
clusters = kmeans.fit_predict(reduced_data)

# Add cluster labels to your dataframe
notes['cluster'] = clusters

# Evaluate Clustering
silhouette_avg = silhouette_score(reduced_data, clusters)
print("Silhouette Score: ", silhouette_avg)

